# Training Batch Shipyard
This notebook will set up everything for the tutorial. This notebook assumes that nothing has been set up previously and will create everything from scratch. The necessary steps are broken up into the following sections:

**Note:** By using these notebooks on GPU VMs, you agree to the [NVIDIA Software License](http://www.nvidia.com/content/DriverDownload-March2009/licence.php?lang=us).

* [Install tools and dependencies](#section1)
* [Azure account login](#section2)
* [Setup](#section3)
* [Create Azure resources](#section4)
* [Define our model](#model)
* [Batch Shipyard Configuration](#section5)
* [Create Azure Batch Pool](#section6)
* [Configure Job](#section7)
* [Submit Job](#section8)
* [Delete Job and Deallocate Pool](#section9)
* [Delete Azure resources](#section10)

<a id='section1'></a>
## Install tools and dependencies
Install Batch Shipyard and Azure CLI 2.0

In [ ]:
!git clone https://github.com/Azure/batch-shipyard.git --branch 3.1.0

Normally you would use `install.sh` or `install.cmd` helper scripts to install, but due to the Notebook environment, we will instead install with the `requirements.txt` file directly.

In [ ]:
import sys

In [ ]:
!{sys.executable} -m pip install blobxfer

In [ ]:
!{sys.executable} -m pip install ruamel.yaml pykwalify==1.6.0

In [ ]:
!{sys.executable} -m pip install -r batch-shipyard/requirements.txt

Azure CLI 2.0 will also be installed to help us in provisioning Azure Batch and Storage accounts.

In [ ]:
!{sys.executable} -m pip install -I azure-cli

We'll create an alias for invoking Batch Shipyard which points to a `config` directory which will hold our yaml config files (this directory will be created at a later step).

In [ ]:
pwd=!pwd
%alias shipyard SHIPYARD_CONFIGDIR=config python {pwd[0]}/batch-shipyard/shipyard.py %l

Check that everything is working:

In [ ]:
shipyard --version

<a id='section2'></a>
## Azure account login
The command below will initiate a login to your Azure account. You will see a url to browse to where you will enter the specified code. This will log you into the Azure account within the Azure CLI.

In [ ]:
!az login -o table

If you have multiple subscriptions you can select the one you need with the command below. This will not be necessary for your assigned Azure Pass account for the workshop.

In [ ]:
selected_subscription = "<YOUR SUBSCRIPTION>" # Replace with the name of your subscription
!az account set --subscription "$selected_subscription"

**Note:** If you cannot login with the Azure CLI, you can create Azure Batch and Storage accounts on the [Azure Portal](https://portal.azure.com).
- [Instructions for creating an Azure Batch Account](https://docs.microsoft.com/en-us/azure/batch/batch-account-create-portal#batch-service-mode)
- [Instructions for creating an Azure Storage Account](https://docs.microsoft.com/en-us/azure/storage/storage-create-storage-account#create-a-storage-account) (You can create an "Auto Storage" account at the same time as your Batch Account on the portal instead)

Please pay attention to special instructions regarding Azure Portal created accounts below.

<a id='section3'></a>
## Setup
First we will need to register the Azure Batch service as a resource provider for the Azure subscription. The following will do so and poll until the registration process has completed. This will take approximately 30 seconds to complete.

**Note:** This registration process needs to be performed only once for the Azure subscription. If you created your Azure Batch account via the Azure Portal, you do not need to perform this action as it is done automatically for you.

In [ ]:
import time

# register resource provider with subscription
print('Registering Microsoft.Batch with subscription. Please be patient for this process.')
!az provider register -n Microsoft.Batch

# poll until registration completed
while True:
    status = !az provider show -n Microsoft.Batch -o table
    if status[-1].split()[-1] == 'Registered':
        print('\n'.join(status))
        break
    time.sleep(2)

Now we will define the various names for the resources needed to run Azure Batch jobs.

**Note:** If you manually created your accounts on the Azure Portal, you will need to modify `GROUP_NAME`, `BATCH_ACCOUNT_NAME` and `STORAGE_ACCOUNT_NAME` accordingly.

In [ ]:
import yaml
import os
import uuid
import random
import json

def write_yaml_to_file(yaml_dict, filename):
    """ Simple function to write YAML dictionaries to files
    """
    with open(filename, 'w') as outfile:
        yaml.dump(yaml_dict, outfile)

LOCATION = 'eastus' # We are setting everything up in East US
                    # Be aware that you need to set things up in a region that has GPU VMs (N-Series)

# Tensorflow image
IMAGE_NAME = "tensorflow/tensorflow:1.8.0-gpu-py3"

short_uuid = str(uuid.uuid4())[:8]
GROUP_NAME = "batch{uuid}rg".format(uuid=short_uuid)
BATCH_ACCOUNT_NAME = "batch{uuid}ba".format(uuid=short_uuid)
STORAGE_ACCOUNT_NAME = "batch{uuid}st".format(uuid=short_uuid)
STORAGE_ALIAS = "mystorageaccount"
STORAGE_ENDPOINT = "core.windows.net"

<a id='section4'></a>
## Create Azure resources
### Create Resource Group
Azure encourages the use of resource groups to organise all the Azure components you deploy. That way it is easier to find them but also we can deleted a number of resources simply by deleting the Resource Group.

In [ ]:
!az group create -n $GROUP_NAME -l $LOCATION -o table

### Create Batch and Storage accounts
Here we simply crate the Batch and Storage accounts. Once we have created the accounts we can the use the Azure CLI to query them and obtain the **batch_account_key**, **batch_service_url** and **storage_account_key** which we will need for our Batch Shipyard configuration files later.

In [ ]:
json_data = !az storage account create -l $LOCATION -n $STORAGE_ACCOUNT_NAME -g $GROUP_NAME --sku Standard_LRS
print('Storage account {} provisioning state: {}'.format(STORAGE_ACCOUNT_NAME, json.loads(''.join(json_data))['provisioningState']))

In [ ]:
json_data = !az batch account create -l $LOCATION -n $BATCH_ACCOUNT_NAME -g $GROUP_NAME --storage-account $STORAGE_ACCOUNT_NAME
print('Batch account {} provisioning state: {}'.format(BATCH_ACCOUNT_NAME, json.loads(''.join(json_data))['provisioningState']))

Next we retrieve the **batch_account_key**, **batch_service_url** and **storage_account_key** which we will need for the Batch Shipyard configuration files further down.

**Note:** If you created your Batch and Storage accounts in the Azure Portal, you will need to retrieve your keys in the Portal. Then set `batch_account_key`, `batch_service_url`, and `storage_account_key` to their appropriate values instead of the Azure CLI callouts.

In [ ]:
json_data = !az batch account keys list -n $BATCH_ACCOUNT_NAME -g $GROUP_NAME
batch_account_key = json.loads(''.join(json_data))['primary']

In [ ]:
json_data = !az batch account list -g $GROUP_NAME
batch_service_url = 'https://'+json.loads(''.join(json_data))[0]['accountEndpoint']

In [ ]:
json_data = !az storage account keys list -n $STORAGE_ACCOUNT_NAME -g $GROUP_NAME
storage_account_key = json.loads(''.join(json_data))[0]['value']

In [ ]:
account_information = {
    "IMAGE_NAME": IMAGE_NAME,
    "LOCATION": LOCATION,
    "resource_group": GROUP_NAME,
    "STORAGE_ALIAS": STORAGE_ALIAS,
    "storage_account_key": storage_account_key,
    "storage_account_name": STORAGE_ACCOUNT_NAME,
}
write_yaml_to_file(account_information, 'account_information.yaml')

<a id='model'></a>
## Define Our Model
The file below contains a simple CNN written in Keras. It will load the CIFAR 10 data and then train the model for a number of epochs and then evaluate it on the test set.

In [ ]:
%%writefile cifar10_cnn.py
'''Train a VGG-like CNN on the CIFAR10 small images dataset.
'''

import numpy as np
import os
import sys
import tarfile
import pickle
import tensorflow as tf
import argparse
import logging
from urllib.request import urlretrieve

# Parameters
EPOCHS = 30
BATCHSIZE = 64
LR = 0.01
MOMENTUM = 0.9
N_CLASSES = 10 # There are 10 classes in the CIFAR10 dataset
DATA_FORMAT = 'channels_first'

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)


def read_pickle(src):
    with open(src, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
    return data


def process_cifar():
    """ Read data
    """
    
    logger.info('Preparing train set...')
    train_list = [read_pickle('./cifar-10-batches-py/data_batch_{0}'.format(i)) for i in range(1, 6)]
    x_train = np.concatenate([t['data'] for t in train_list])
    y_train = np.concatenate([t['labels'] for t in train_list])
    
    logger.info('Preparing test set...')
    tst = read_pickle('./cifar-10-batches-py/test_batch')
    x_test = tst['data']
    y_test = np.asarray(tst['labels'])
    
    return x_train, y_train, x_test, y_test


def prepare_cifar(x_train, y_train, x_test, y_test):
    
    # Scale pixel intensity
    x_train = x_train / 255.0
    x_test = x_test / 255.0
    
    # Reshape
    x_train = x_train.reshape(-1, 3, 32, 32)
    x_test = x_test.reshape(-1, 3, 32, 32)
    
    return (x_train.astype(np.float32), 
            y_train.astype(np.int32), 
            x_test.astype(np.float32), 
            y_test.astype(np.int32))


def load_cifar(src="http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"):
    """ Load CIFAR10 Dataset
    """
    try:
        return process_cifar()
    except FileNotFoundError:
        logger.info('Data does not exist. Downloading ' + src)
        fname, h = urlretrieve(src, './delete.me')
        logger.info('Extracting files...')
        with tarfile.open(fname) as tar:
            tar.extractall()
        os.remove(fname)
    return process_cifar()


def init_model_training(m, labels, learning_rate=LR, momentum=MOMENTUM):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=m, labels=labels)
    loss = tf.reduce_mean(cross_entropy)
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=momentum)
    return optimizer.minimize(loss)


def shuffle_data(X, y):
    index = np.arange(len(X))
    np.random.shuffle(index)
    return X[index], y[index]


def minibatch_from(X, y, batchsize=BATCHSIZE, shuffle=False):
    if len(X) != len(y):
        raise Exception("The length of X {} and y {} don't match".format(len(X), len(y)))
        
    if shuffle:
        X, y = shuffle_data(X, y)
    
    for i in range(0, len(X), batchsize):
        yield X[i:i + batchsize], y[i:i + batchsize]
        

def main(epochs=EPOCHS, lr=LR, mb_size=BATCHSIZE, data_format=DATA_FORMAT):
    logger.info('Learning Rate: {} Minibatch Size: {} Epochs: {}'.format(lr, mb_size, epochs))
    
    logger.info('Loading data....')
    # Data into format for library
    x_train, y_train, x_test, y_test = prepare_cifar(*load_cifar())
    logger.info('Data shape {}'.format(str((x_train.shape, x_test.shape, y_train.shape, y_test.shape))))
    logger.info('Data types {}'.format(str((x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype))))
    
    tf.reset_default_graph()
    # Place-holders
    X = tf.placeholder(tf.float32, shape=[None, 3, 32, 32])
    y = tf.placeholder(tf.int32, shape=[None])
    training = tf.placeholder(tf.bool)  # Indicator for dropout layer
    
    # Define model
    # Block 1
    conv1_1 = tf.layers.conv2d(X, 
                               filters=64, 
                               kernel_size=(3, 3), 
                               padding='same', 
                               data_format=data_format,
                               activation=tf.nn.relu)
    conv1_2 = tf.layers.conv2d(conv1_1, 
                               filters=64, 
                               kernel_size=(3, 3), 
                               padding='same', 
                               data_format=data_format,
                               activation=tf.nn.relu)
    pool1_1 = tf.layers.max_pooling2d(conv1_2, 
                                      pool_size=(2, 2), 
                                      strides=(2, 2), 
                                      padding='valid', 
                                      data_format=data_format)
    # Block 2
    conv2_1 = tf.layers.conv2d(pool1_1, 
                               filters=128, 
                               kernel_size=(3, 3), 
                               padding='same', 
                               data_format=data_format,
                               activation=tf.nn.relu)
    conv2_2 = tf.layers.conv2d(conv2_1, 
                               filters=128, 
                               kernel_size=(3, 3), 
                               padding='same', 
                               data_format=data_format,
                               activation=tf.nn.relu)
    pool2_1 = tf.layers.max_pooling2d(conv2_2, 
                                      pool_size=(2, 2), 
                                      strides=(2, 2), 
                                      padding='valid', 
                                      data_format=data_format)

    # Block 3
    conv3_1 = tf.layers.conv2d(pool2_1, 
                               filters=256, 
                               kernel_size=(3, 3), 
                               padding='same', 
                               data_format=data_format,
                               activation=tf.nn.relu)
    conv3_2 = tf.layers.conv2d(conv3_1, 
                               filters=256, 
                               kernel_size=(3, 3), 
                               padding='same', 
                               data_format=data_format,
                               activation=tf.nn.relu)
    conv3_3 = tf.layers.conv2d(conv3_2, 
                               filters=256, 
                               kernel_size=(3, 3), 
                               padding='same', 
                               data_format=data_format,
                               activation=tf.nn.relu)
    pool3_1 = tf.layers.max_pooling2d(conv3_3, 
                                      pool_size=(2, 2), 
                                      strides=(2, 2), 
                                      padding='valid', 
                                      data_format=data_format)

    relu2 = tf.nn.relu(pool3_1)
    flatten = tf.reshape(relu2, shape=[-1, 256*4*4])
    fc1 = tf.layers.dense(flatten, 4096, activation=tf.nn.relu)
    drop1 = tf.layers.dropout(fc1, 0.5, training=training)
    fc2 = tf.layers.dense(drop1, 4096, activation=tf.nn.relu)
    drop2 = tf.layers.dropout(fc2, 0.5, training=training)
    model = tf.layers.dense(drop2, N_CLASSES, name='output')

    train_model = init_model_training(model, y, learning_rate=lr)
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)

    # Accuracy logging
    correct = tf.nn.in_top_k(model, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    logger.info('Training model...')
    
    # Train model
    for j in range(epochs):
        for data, label in minibatch_from(x_train, y_train, shuffle=True, batchsize=mb_size):
            sess.run(train_model, feed_dict={X: data, y: label, training: True})
        # Log
        acc_train = sess.run(accuracy, feed_dict={X: data, y: label, training: True})
        logger.info("{} | Train accuracy: {}".format(j, acc_train))
    
    logger.info('Evaluating model...')
    y_guess = list()
    for data, label in minibatch_from(x_test, y_test):
        pred = tf.argmax(model,1)
        output = sess.run(pred, feed_dict={X: data, training: False})
        y_guess.append(output)
    logger.info("Accuracy: {}".format(sum(np.concatenate(y_guess) == y_test)/float(len(y_test))))
    
    
if __name__=='__main__':
    logger.info('Starting script....')
    parser = argparse.ArgumentParser(description='Script to train VGG-like model on CIFAR10 dataset')
    parser.add_argument('--lr', help='Specify learning rate', type=float, default=LR)
    parser.add_argument('--mb_size', help='Minibatch size', type=int, default=BATCHSIZE)
    parser.add_argument('--epochs', help='Number of epochs to train for', type=int, default=EPOCHS)
    args = parser.parse_args()
    main(epochs=args.epochs, lr=args.lr, mb_size=args.mb_size)
    

<a id='section5'></a>
## Batch Shiyard configuration
In order to execute a job on Batch Shipyard you need a minimum of four configuration files. We will set three of them here and leave the job one for later.
* [credentials](#credentials)
* [configuration](#configuration)
* [pool](#pool)

In [ ]:
INPUT_CONTAINER = 'input'
UPLOAD_DIR = 'dist_upload'

!rm -rf $UPLOAD_DIR
!mkdir -p $UPLOAD_DIR
!mv cifar10_cnn.py $UPLOAD_DIR
!chmod 777 $UPLOAD_DIR/cifar10_cnn.py
!ls -alF $UPLOAD_DIR

<a id='credentials'></a>
### Credentials
Here we define all the credentials necessary for Batch Shipyard to connect to Azure for resource provisioning and executing our jobs.

In [ ]:
credentials = {
    "credentials": {
        "batch": {
            "account_key": batch_account_key,
            "account_service_url": batch_service_url
        },
        "storage": {
            STORAGE_ALIAS : {
                    "account": STORAGE_ACCOUNT_NAME,
                    "account_key": storage_account_key,
                    "endpoint": STORAGE_ENDPOINT
            }
        }
    }
}

<a id='configuration'></a>
### Configuration
The config mainly contains the configuration for Batch Shipyard. Here we simply define the storage alias that Batch Shipyard should use as well as the Docker image to use. We also tell Batch shipyard to upload everything it finds in the `dist_upload` directory to the `input` blob container. The `dist_upload` directory contains the script we wrote earlier which we train and evaluate our deep learning model.

In [ ]:
config = {
    "batch_shipyard": {
        "storage_account_settings": STORAGE_ALIAS,
    },
   "global_resources": {
        "docker_images": [
            IMAGE_NAME
        ],
        "files": [
            {
                "source": {
                    "path": "dist_upload"
                },
                "destination": {
                    "storage_account_settings": STORAGE_ALIAS,
                    "data_transfer": {
                        "remote_path": INPUT_CONTAINER
                    }
                }
            }
        ]
    }
}

<a id='pool'></a>
### Pool
This is where we define the properties of the compute pool we wish to create. The configuration below creates a pool that is made up of a single NC6 VM running Ubuntu 16.04. If you wish to run a job that uses GPU accelerated compute, as we will be doing for these notebooks, then you will need to choose a VM from the NC series. Here we will allocate 1 `STANDARD_NC6` instances. You may opt to change the `vm_count` from `dedicated` to `low_priority` to save on costs, but please note that you may be pre-empted at any time. It is recommended to use `dedicated` for this tutorial to avoid running into those issues.

In [ ]:
POOL_ID = 'gpupool'

pool = {
    "pool_specification": {
        "id": POOL_ID,
        "vm_configuration": {
            "platform_image": {
                "publisher": "Canonical",
                "offer": "UbuntuServer",
                "sku": "16.04-LTS",
                "native": False,
            },
        },
        "vm_size": "STANDARD_NC6",
        "vm_count": {
            "dedicated": 1
        },
        "ssh": {
            "username": "shipyard",
            "generate_docker_tunnel_script": True
        },
        "reboot_on_start_task_failed": False,
        "block_until_all_global_resources_loaded": True,
        "transfer_files_on_pool_creation": True,
    }
}

In [ ]:
!mkdir -p config # Create config file directory where we will store all our Batch Shipyard configuration files

In [ ]:
write_yaml_to_file(credentials, os.path.join('config', 'credentials.yaml'))

In [ ]:
write_yaml_to_file(config, os.path.join('config', 'config.yaml'))

In [ ]:
write_yaml_to_file(pool, os.path.join('config', 'pool.yaml'))

In [ ]:
print('IMAGE_NAME = "{}"'.format(IMAGE_NAME))
print('GROUP_NAME = "{}"'.format(GROUP_NAME))
print('LOCATION = "{}"'.format(LOCATION))
print('BATCH_ACCOUNT_NAME = "{}"'.format(BATCH_ACCOUNT_NAME))
print('batch_account_key = "{}"'.format(batch_account_key))
print('batch_service_url = "{}"'.format(batch_service_url))
print('STORAGE_ACCOUNT_NAME = "{}"'.format(STORAGE_ACCOUNT_NAME))
print('STORAGE_ALIAS = "{}"'.format(STORAGE_ALIAS))
print('storage_account_key = "{}"'.format(storage_account_key))

<a id='section6'></a>
## Create Azure Batch Pool
Before we do anything we need to create the pool for Batch Shipyard jobs to run on. This can take a little bit of time so please be patient while the compute nodes are allocated from the Azure Cloud and the Docker images are pre-loaded on to the compute nodes.

In [ ]:
shipyard pool add -y

Once the pool is created we can confirm everything by running the command below.

In [ ]:
shipyard pool list

<a id='section7'></a>
## Configure Job
As before the dictionary below defines the job we will execute. Here we are pulling the script from the blob container that we uploaded during pool creation into `AZ_BATCH_TASK_WORKING_DIR` and executing it.

In [ ]:
TASK_ID = 'run_cifar10' # This should be changed per task
JOB_ID = 'tf-training-job'
COMMAND = '/bin/bash -c "python cifar10_cnn.py"'

jobs = {
    "job_specifications": [
        {
            "id": JOB_ID,
            "allow_run_on_missing_image": False,
            "gpu": True,
            "tasks": [
                {
                    "id": TASK_ID,
                    "docker_image": IMAGE_NAME,
                    "command": COMMAND,
                    "gpu": True,
                    "input_data": {
                    "azure_storage": [
                        {
                            "storage_account_settings": STORAGE_ALIAS,
                            "remote_path": INPUT_CONTAINER,
                            "local_path": "$AZ_BATCH_TASK_WORKING_DIR"
                        }
                    ]
                },
                }
            ],
        }
    ]
}

In [ ]:
write_yaml_to_file(jobs, os.path.join('config', 'jobs.yaml'))
print(yaml.dump(jobs))

<a id='section8'></a>
## Submit Job
Now that we have confirmed everything is working we can execute our job using the command below. The tail switch at the end will stream stdout from the node.

In [ ]:
shipyard jobs add --tail stdout.txt

We can stream any of the text files that are on the node.

In [ ]:
shipyard data files stream -v --filespec $JOB_ID,$TASK_ID,stderr.txt

In [ ]:
shipyard jobs list

<a id='section9'></a>
## Delete job and deallocate pool

In [ ]:
shipyard jobs del -y --termtasks --wait

In [ ]:
shipyard pool del -y --wait

<a id='section10'></a>
## Delete Azure resources
Once you have deleted the pool all that remains is the storage account and the Batch account.

Note that you do not need to delete your batch and storage accounts.
- You will only be billed in Batch for pools for compute node time and data egress. If you do not have any active pools with nodes in them, you will not be billed for anything.
- Storage costs include data stored in blobs and transactions. For the examples in these notebooks, the cost will be very small.

However, if you wish to delete your accounts, you can do so by deleting the resource group containing the accounts.

In [ ]:
!az group delete -n $GROUP_NAME --yes --verbose